In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

# Heading

```python
from pyspark import SparkContext, SparkConf
from user_definition import *
#Do not add additional libraries.

conf = SparkConf().setMaster("local[*]").setAppName(app_name)
sc = SparkContext(conf=conf)
```

In [3]:
# from pyspark import SparkContext, SparkConf
from user_definition import *
#Do not add additional libraries.

# conf = SparkConf().setMaster("local[*]").setAppName(app_name)
# sc = SparkContext(conf=conf)

### Fold

In [15]:
lines = sc.parallelize([1,2,3,4,5,6,7,8,9], 3)
lines.glom().collect()

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

In [16]:
lines.fold(0, lambda x,y: x+y)

45

### Aggregate Odd numbers sum and Count - note this is for single values

In [26]:
lines = sc.parallelize([1,2,3,4,5,6,7,8,9], 3)
odds = lines.filter( lambda x : x % 2 == 1)
odds = odds.map(lambda x : float(x))
print odds.collect()
res = odds.aggregate((0,0),
               lambda x,y: (x[0]+1, x[1]+y),
               lambda x,y: (x[0]+y[0], x[1] + y[1])
              )
print res

[1.0, 3.0, 5.0, 7.0, 9.0]
(5, 25.0)


In [29]:
odd_nums = sc.parallelize([1,3,5,7,9],2)

In [33]:
odd_nums.take(1)

[1]

In [31]:
odd_nums.first()

1

### Take Sample and CountByValue

In [34]:
x = sc.parallelize([3,4,1,2])
y = sc.parallelize(range(2,6))
z = x.union(y)

In [35]:
z.countByValue()


defaultdict(int, {1: 1, 2: 2, 3: 2, 4: 2, 5: 1})

In [36]:
z.takeSample(withReplacement=False,num=3,seed=None)

[1, 2, 2]

In [37]:
path = '/Users/tlee010/Desktop/github_repos/2017-msan694-example/Data/README.md'
rdm = sc.textFile(path)

### Example Keys and Values

In [38]:
words = rdm.flatMap(lambda x: x.split())
words = words.map(lambda x: (x,1))
words.collect()[:20]

[(u'#', 1),
 (u'Apache', 1),
 (u'Spark', 1),
 (u'Spark', 1),
 (u'is', 1),
 (u'a', 1),
 (u'fast', 1),
 (u'and', 1),
 (u'general', 1),
 (u'cluster', 1),
 (u'computing', 1),
 (u'system', 1),
 (u'for', 1),
 (u'Big', 1),
 (u'Data.', 1),
 (u'It', 1),
 (u'provides', 1),
 (u'high-level', 1),
 (u'APIs', 1),
 (u'in', 1)]

In [41]:
words.keys().collect()[:5]

[u'#', u'Apache', u'Spark', u'Spark', u'is']

In [43]:
words.values().collect()[:5]

[1, 1, 1, 1, 1]

### Group By
Create a pair RDD with (length of a word, list of words) from README.md

In [46]:
words = rdm.flatMap(lambda x: x.split())
words = words.map(lambda x: (len(x),x))
res = words.groupByKey().map(lambda x: (x[0],list(x[1])))

for x in res.collect()[:3]:
    print x[:2]
    print '----------------'

(96, [u'Version"](http://spark.apache.org/docs/latest/building-spark.html#specifying-the-hadoop-version)'])
----------------
(112, [u'[Eclipse](https://cwiki.apache.org/confluence/display/SPARK/Useful+Developer+Tools#UsefulDeveloperTools-Eclipse)'])
----------------
(2, [u'is', u'It', u'in', u'R,', u'an', u'It', u'of', u'##', u'on', u'##', u'is', u'To', u'do', u'to', u'do', u'if', u'by', u'-T', u'in', u'is', u'at', u'an', u'##', u'to', u'is', u'to', u'##', u'if', u'##', u'in', u'To', u'of', u'Pi', u'to', u'to', u'be', u'or', u'to', u'on', u'to', u'or', u'to', u'an', u'if', u'is', u'in', u'of', u'if', u'no', u'##', u'is', u'be', u'on', u'to', u'or', u'##', u'to', u'to', u'in', u'of', u'to', u'at', u'on', u'of', u'##', u'to', u'in', u'an', u'on', u'to'])
----------------


### MapValues

In [48]:
words = rdm.flatMap(lambda x: x.split())
words = words.map(lambda x: (x,1))
res = words.groupByKey()
res2 = res.mapValues(sum)
res2.sortBy(lambda x: x[1], ascending = False).collect()[:10]

[(u'the', 22),
 (u'Spark', 15),
 (u'to', 14),
 (u'for', 11),
 (u'and', 11),
 (u'##', 8),
 (u'a', 8),
 (u'run', 7),
 (u'can', 7),
 (u'is', 6)]

### flatMapValues

In [49]:
words = rdm.flatMap(lambda x: x.split())
words = words.map(lambda x: (len(x),x))
words.mapValues(lambda x : list([x])).collect()[:10]

[(1, [u'#']),
 (6, [u'Apache']),
 (5, [u'Spark']),
 (5, [u'Spark']),
 (2, [u'is']),
 (1, [u'a']),
 (4, [u'fast']),
 (3, [u'and']),
 (7, [u'general']),
 (7, [u'cluster'])]

### reduceByKey

In [ ]:
words = rdm.flatMap(lambda x: x.split())
words = words.map(lambda x: (x,1))
res = words.reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], ascending=False)
res.collect()[:10]

### CombineByKey

In [51]:
PATH = '/Users/tlee010/Desktop/github_repos/2017-msan694-example/Data/README.md'
lines = sc.textFile(PATH)
words = lines.flatMap(lambda x : x. split(' '))
words = words.map(lambda x : (len(x),x))

comp = words.combineByKey(
                    (lambda x : (1,x)),  # sets a default (new nested value)
                   (lambda x,y : (x[0] +1 , x[1] +', '+ y)), # appends 
                   (lambda x,y : (x[0] + y[0], x[1] +', '+y[1]))) # adds all the lists together across partitions
comp.take(5)

[(0,
  (68,
   u', , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ')),
 (112,
  (1,
   u'[Eclipse](https://cwiki.apache.org/confluence/display/SPARK/Useful+Developer+Tools#UsefulDeveloperTools-Eclipse)')),
 (2,
  (70,
   u'is, It, in, R,, an, It, of, ##, on, ##, is, To, do, to, do, if, by, -T, in, is, at, an, ##, to, is, to, ##, if, ##, in, To, of, Pi, to, to, be, or, to, on, to, or, to, an, if, is, in, of, if, no, ##, is, be, on, to, or, ##, to, to, in, of, to, at, on, of, ##, to, in, an, on, to')),
 (4,
  (47,
   u'fast, APIs, that, data, also, rich, find, This, file, only, run:, (You, need, this, more, than, with, More, from, IDE,, also, also, with, will, when, This, URL,, with, with, also, name, Many, help, Once, [run, Note, uses, core, talk, HDFS, have, must, same, that, your, Hive, Hive')),
 (6,
  (41,
   u'Apache, system, Scala,, engine, graphs, GraphX, stream, Online, latest, guide,, README, th

### SubtractByKey

In [53]:

PATH1 = '/Users/tlee010/Desktop/github_repos/2017-msan694-example/Data/supervisor_sf.csv'
PATH2 = '/Users/tlee010/Desktop/github_repos/2017-msan694-example/Data/filtered_registered_business_sf.csv'

In [54]:
lines1 = sc.textFile(PATH1)
lines2 = sc.textFile(PATH2)

In [55]:
# split by word
zippairs = lines1.map(lambda x : x.split(','))

# then turn into tuple 
zippairs = zippairs.map(lambda x : (x[0],x[1]))

# look at data
zippairs.take(5)

[(u'94102', u'8'),
 (u'94102', u'6'),
 (u'94102', u'3'),
 (u'94102', u'5'),
 (u'94103', u'8')]

In [56]:
# split the 2nd file by commas
zipbus = lines2.map(lambda x : x.split(','))

# split the 2nd file by commas
zipbus = zipbus.map(lambda x : (x[0], x[1]))
zipbus.take(5)

[(u'94123', u'Tournahu George L'),
 (u'94124', u'Stephens Institute Inc'),
 (u'94105', u'Stephens Institute Inc'),
 (u'94108', u'Stephens Institute Inc'),
 (u'94107', u'Stephens Institute Inc')]

In [57]:
res = zipbus.subtractByKey(zippairs).values().distinct()
res.take(5)

[u'Precision Communication Serv',
 u'Schefer Thomas R',
 u'Tyrrell Thomas',
 u'Lucid Systems',
 u'Jacob Abraham']

### Joins

In [63]:
first_num_pairs = sc.parallelize({(2,3), (1,2),(1,3),(2,4),(3,6)})

second_num_pairs = sc.parallelize({(1,3),(2,2),(2,10)})

### Subtract (both key and value)

In [64]:
first_num_pairs.subtract(second_num_pairs).collect()


[(2, 3), (3, 6), (1, 2), (2, 4)]

### Subtract by Key (key only match)

In [65]:
first_num_pairs.subtractByKey(second_num_pairs).collect()


[(3, 6)]

### Join

In [66]:
first_num_pairs.join(second_num_pairs).collect()

[(1, (2, 3)),
 (1, (3, 3)),
 (2, (3, 10)),
 (2, (3, 2)),
 (2, (4, 10)),
 (2, (4, 2))]

## Right outer join

In [67]:
first_num_pairs.rightOuterJoin(second_num_pairs).collect()

[(1, (2, 3)),
 (1, (3, 3)),
 (2, (3, 10)),
 (2, (3, 2)),
 (2, (4, 10)),
 (2, (4, 2))]

### Left Outer Join

In [68]:
first_num_pairs.leftOuterJoin(second_num_pairs).collect()


[(1, (2, 3)),
 (1, (3, 3)),
 (2, (3, 10)),
 (2, (3, 2)),
 (2, (4, 10)),
 (2, (4, 2)),
 (3, (6, None))]

### Cogroup - no cartesianing

In [73]:
first_num_pairs.cogroup(second_num_pairs).mapValues(lambda x: (sum(x[0]),sum(x[1]))).collect()

[(1, (5, 3)), (2, (7, 12)), (3, (6, 0))]

## Lookup

In [76]:
superzips = lines1.map(lambda x: x.split(','))
superzips = superzips.map(lambda x : (x[0],x[1]))
superzips.take(5)

[(u'94102', u'8'),
 (u'94102', u'6'),
 (u'94102', u'3'),
 (u'94102', u'5'),
 (u'94103', u'8')]

In [77]:
businesszips = lines2.map(lambda x: x.split(','))
businesszips = businesszips.map(lambda x : (x[0],x[1]))
businesszips.take(5)

[(u'94123', u'Tournahu George L'),
 (u'94124', u'Stephens Institute Inc'),
 (u'94105', u'Stephens Institute Inc'),
 (u'94108', u'Stephens Institute Inc'),
 (u'94107', u'Stephens Institute Inc')]

In [78]:
superzips.cogroup(businesszips).take(6)

[(u'',
  (<pyspark.resultiterable.ResultIterable at 0x10fcf97d0>,
   <pyspark.resultiterable.ResultIterable at 0x10fcf9250>)),
 (u'85716',
  (<pyspark.resultiterable.ResultIterable at 0x10fcf9f10>,
   <pyspark.resultiterable.ResultIterable at 0x10fcf9f50>)),
 (u'28036',
  (<pyspark.resultiterable.ResultIterable at 0x10fcf9f90>,
   <pyspark.resultiterable.ResultIterable at 0x10fcf9fd0>)),
 (u'97302',
  (<pyspark.resultiterable.ResultIterable at 0x10fcf6050>,
   <pyspark.resultiterable.ResultIterable at 0x10fcf6090>)),
 (u'55344',
  (<pyspark.resultiterable.ResultIterable at 0x10fcf6150>,
   <pyspark.resultiterable.ResultIterable at 0x10fcf6190>)),
 (u'94035',
  (<pyspark.resultiterable.ResultIterable at 0x10fcf61d0>,
   <pyspark.resultiterable.ResultIterable at 0x10fcf6210>))]

In [79]:
superzips.countByKey()

defaultdict(int,
            {u'94102': 4,
             u'94103': 6,
             u'94104': 2,
             u'94105': 2,
             u'94107': 2,
             u'94108': 2,
             u'94109': 4,
             u'94110': 4,
             u'94111': 2,
             u'94112': 5,
             u'94114': 3,
             u'94115': 3,
             u'94116': 2,
             u'94117': 4,
             u'94118': 3,
             u'94121': 2,
             u'94122': 4,
             u'94123': 1,
             u'94124': 2,
             u'94127': 3,
             u'94129': 1,
             u'94130': 1,
             u'94131': 3,
             u'94132': 3,
             u'94133': 2,
             u'94134': 3,
             u'94158': 2})

### Super Hard Practice Quiz

```python
from pyspark import SparkContext, SparkConf
from user_definition import *
# Do not add any additional libraries.

conf = SparkConf().setMaster("local[*]").setAppName(app_name)
sc = SparkContext(conf=conf)

lines = sc.textFile('input.csv')
lines.collect()

nums = lines.flatMap(lambda x : x.split(','))
nums = nums.map(lambda x : int(x))

evens = nums.filter(lambda x : x % 2 == 0)
evens = evens.sortBy(lambda x : x)
max_even = evens.max()
min_even = evens.min()

even_partitions = range(min_even, max_even, (max_even-min_even)/hist_size )

odds = nums.filter(lambda x : x % 2 == 1)
odds = odds.sortBy(lambda x: x)
max_odd = odds.max()
min_odd = odds.min()

odd_partitions = range(min_odd, max_odd, (max_odd-min_odd)/hist_size)

def assign_even_key(x):
    for y in even_partitions[::-1]:
        if  x >= y:
            return (y,1)
        
def assign_odd_key(x):
    for y in odd_partitions[::-1]:
        if  x >= y:
            return (y,1)

histdodds = odds.map(assign_odd_key)
histevens = evens.map(assign_even_key)

even_res = dict(histevens.reduceByKey(lambda x, y : x + y).collect())
odd_res = dict(histdodds.reduceByKey(lambda x, y : x + y).collect())

for x in odd_partitions:
    if x not in odd_res:
        odd_res[x] =0
        
for x in even_partitions: 
    if x not in even_res:
        even_res[x] =0
print (even_partitions, [even_res[x] for x in even_partitions])
print (odd_partitions, [odd_res[x] for x in odd_partitions])
```




### High Level Concepts


    - keys()
    - values()
    - sortByKey()
    - sortBy(lambda x : x[1]) <- can pass other params here to sort
    - groupByKey() - > gives key, iterable
    - mapValues(func) does not change keys (keeps original partitioning)
    - flatMapvalues(func) returns an iterator for each k-v pair,
    - reduceByKey(func) - combines values with same key like reduce (horizontal) TRANSFORMATION new RDD
    - combineBykey(default_value, in-partition func, across partition_func) - 
    
    - groupByKey searches over partitions, reduce by key does not
    
    - subtrackByKey
    - join
    - rightOuterJoin
    - leftOuterJoin
    - cogroup -> can work on more than 1 RDD at once
    
    - countByKey()
    - lookup(key)
    

### Chunks

In [83]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        
[x for x in chunks([1,2,3,4,5,6,7,8,9,10],3)]

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10]]